In [ ]:
import requests
import time
import sqlite3

# 카테고리 설정
# 카테고리별 카테고리 id값 설정
# 이케아에서 지정한 페이로드값에 따라 커테고리를 2종류로 분류

# 1. 상세 / 대형 이미지형 (2열, 5필터)
LARGE_IMAGE_CATEGORIES = {
    "st003": "수납장/장식장",
    "st002": "책장/선반유닛",
    "19053": "옷장"
}

# 2. 콤팩트 / 다수 상품형 (4열, 3필터) - 총 44개 카테고리
COMPACT_PRODUCT_CATEGORIES = {
    "700292": "주방 캐비닛",
    "22957": "미니 주방",
    "10471": "아일랜드 식탁/주방 카트",
    "ka002": "주방 가전",
    "24264": "주방 조리대",
    "16200": "주방 팬트리",
    "ka003": "주방 시스템",
    "700424": "오피스 책상/의자 세트",
    "47068": "회의실 의자",
    "55002": "게임용 가구",
    "54173": "회의 테이블",
    "53249": "책상/의자 세트",
    "20652": "의자/사무실 의자",
    "20649": "책상/컴퓨터 책상",
    "59250": "화장대 의자/스툴",
    "45782": "영유아 의자",
    "bc004": "어린이 의자",
    "18768": "어린이 테이블",
    "20611": "스텝 스툴/스텝 의자",
    "700319": "벤치",
    "19141": "카페 가구",
    "22659": "스툴",
    "16244": "바 테이블/의자",
    "10705": "커피테이블/사이드테이블",
    "700676": "의자",
    "700675": "테이블",
    "700417": "다이닝 가구",
    "57527": "긴 의자/카우치",
    "20926": "풋스툴/발받침대",
    "10663": "소파침대/소파베드",
    "fu006": "암체어/안락의자",
    "fu003": "소파",
    "54992": "침실 가구 세트",
    "700513": "매트리스 포함 침대",
    "20656": "침대 협탁",
    "bm003": "침대/침대프레임",
    "10456": "신발장",
    "10385": "수납유닛/수납장",
    "20474": "장난감 수납/정리함",
    "700411": "복도 가구 세트",
    "46080": "칸막이/파티션",
    "st004": "서랍장/침대협탁",
    "46052": "수납 솔루션 시스템",
    "30454": "거실장/찬장/콘솔테이블",
    "fu005": "트롤리/미니 카트",
    "10475": "TV/멀티미디어 가구",
    "21958": "야외용 수납가구",
    "700440": "창고 수납",
}

# 2. 기본 설정
# 이케아 가구 이미지 api URL
API_URL = 'https://sik.search.blue.cdtapps.com/kr/ko/search?c=listaf&v=20250507'
# 페이지당 상품 개수(이케아에서 지정한 값을 따름)
PRODUCT_SIZE = 24 
# 생성할 SQLite DB 파일 이름
DATABASE_NAME = "ikea_image_data_multi_category2.db" 

# Request Payload 설정
BASE_PAYLOAD_TEMPLATE = {
    "searchParameters": {
        "input": "", # 카테고리 ID가 동적으로 삽입될 위치
        "type": "CATEGORY"
    },
    "isUserLoggedIn": False,
    "components": [
        {
            "component": "PRIMARY_AREA",
            "columns": 4, # 카테고리 그룹에 따라 동적으로 변경됨
            "allVariants": False,
            "filterConfig": {
                "max-num-filters": 3 # 카테고리 그룹에 따라 동적으로 변경됨
            },
            "forceFilterCalculation": True,
            "types": {
                "main": "PRODUCT",
                "breakouts": ["PLANNER", "LOGIN_REMINDER", "MATTRESS_WARRANTY", "FINANCIAL_SERVICES"]
            },
            "window": {
                "size": PRODUCT_SIZE, 
                "offset": 0  # 페이지네이션을 위해 반복문에서 변경될 값 
            }
        }
    ]
}

# 파일 이름 처리 함수
def clean_filename(alt_text):
    """파일 이름으로 사용할 수 없거나 불필요한 문자를 제거하고 공백을 밑줄로 대체"""
    # Windows/Linux/Mac에서 파일명으로 사용할 수 없는 문자를 제거
    invalid_chars = '<>:"/\\|?*\n'
    for char in invalid_chars:
        alt_text = alt_text.replace(char, '')
    # 파일명 앞뒤의 공백을 제거하고 공백을 밑줄로 대체
    return alt_text.strip().replace(' ', '_')

# 데이터베이스 설정 함수
def setup_database(db_name):
    """
    SQLite 데이터베이스에 연결하고 필요한 테이블을 생성
    *image_url*에 UNIQUE 제약 조건을 유지하고 *color* 컬럼을 추가
    """
    try:
        # sqlite db파일 생성
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        print(f"데이터베이스 '{db_name}' 연결 성공. 테이블 구조를 확인")

        # 상품 정보를 저장할 테이블 생성
        # prooducts_images 테이블
        # id: 고유한 id 넘버
        # product_name: 제품의 브랜드(예: KIVIK 쉬비크, SÖDERHAMN 쇠데르함 등)
        # filename: 구체적인 제품 이름(예: SÖDERHAMN_쇠데르함_4인용소파,_긴의자탈뮈라_다크그린)
        # image_url: 가구 이미지의 url
        # category_id: 가구의 카테고리 id(크롤링에 사용된 카테고리 id값)
        # category_name: 카테고리 이름 
        # color: 제품의 색깔(같은 제품에서 다른 색깔을 가진 경우 구분)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS products_images (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                product_name TEXT NOT NULL,
                filename TEXT NOT NULL UNIQUE,
                image_url TEXT NOT NULL UNIQUE, 
                category_id TEXT, 
                category_name TEXT,
                color TEXT
            )
        """)

        # 테이블 생성 명령 처리
        conn.commit()
        return conn, cursor
    except sqlite3.Error as e:
        print(f"[오류] SQLite 연결 또는 테이블 생성 실패: {e}")
        return None, None

# 데이터베이스내 데이터 중복 확인 및 처리 함수
def check_and_get_unique_filename(cursor, base_filename):
    """
    주어진 기본 파일명에 대해 데이터베이스에서 파일명 탐색
    (DB를 직접 쿼리하여 중복 처리, 기존 파일명과 충돌하지 않는 파일명을 반환)
    """
    final_filename = base_filename
    count = 1
    
    while True:
        # 데이터베이스에서 현재 파일명이 존재하는지 확인
        cursor.execute("SELECT 1 FROM products_images WHERE filename = ?", (final_filename,))
        if cursor.fetchone() is None:
            # 존재하지 않으면 이 파일명을 그대로 사용
            return final_filename
        
        # 존재하면 카운트를 증가시키고 새로운 파일명 지정
        count += 1
        # 파일명 중복 방지를 위해 _2 의 형식으로 이름 지정
        final_filename = f"{base_filename}_{count}"

# 이케아 가구 데이터 db 저장 함수
def process_product_data(product_data, cursor, category_id, category_name_kor):
    """
    상품 데이터와 변형 상품 데이터에서 대표 이미지 URL 및 관련 정보를 추출하고 데이터베이스에 저장
    """

    # 성공적으로 DB에 삽입된 아이템 수
    items_processed_success = 0 

    try:
        
        results_list = product_data.get('results', [])
        if not results_list:
            return 0

        # api 호출을 통해 받아온 데이터에서 가구 정보 추출
        first_result = results_list[0]
        items = first_result.get('items', [])

        # 1. 상품(items) 순회
        for item_index, item in enumerate(items):
            product = item.get('product')
            if not product:
                continue
            
            # 2. 처리할 이미지 데이터 목록 생성: 메인 상품 + 모든 변형 상품
            images_to_process = []
            
            # 2-A. 메인 상품 데이터 추가
            # 메인 상품의 색상 정보는 product.validDesignText에서 추출
            images_to_process.append({
                'name': product.get('name', '이름_없음'),
                'alt': product.get('mainImageAlt'),
                'url': product.get('mainImageUrl'),
                'color': product.get('validDesignText', '색상_없음')
            })
            
            # 2-B. 변형 상품(Variants) 데이터 추가
            gpr_description = product.get('gprDescription', {})
            variants = gpr_description.get('variants', [])
            
            for variant in variants:
                # 변형 상품의 색상 정보는 variant.validDesignText에서 추출
                images_to_process.append({
                    'name': variant.get('name', product.get('name', '이름_없음')), 
                    'alt': variant.get('mainImageAlt'),
                    'url': variant.get('mainImageUrl'),
                    'color': variant.get('validDesignText', '색상_없음')
                })

            # 3. 이미지 목록 순회 및 DB 저장
            for img_data in images_to_process:
                product_name = img_data['name']
                alt_text = img_data['alt']
                url = img_data['url']
                color = img_data['color']
                
                if not alt_text or not url:
                    continue

                # 파일명 생성 및 고유성 확보
                base_filename_initial = clean_filename(alt_text)
                final_filename = check_and_get_unique_filename(cursor, base_filename_initial)

                # 데이터베이스에 저장
                try:
                    cursor.execute("""
                        INSERT INTO products_images (product_name, filename, image_url, category_id, category_name, color)
                        VALUES (?, ?, ?, ?, ?, ?)
                    """, (product_name, final_filename, url, category_id, category_name_kor, color))
                    
                    # 성공적으로 저장된 db 행 개수 확인을 위한 변수
                    items_processed_success += 1
                    
                except sqlite3.IntegrityError as e:
                    # 동일한 이미지 URL이 이미 존재할 경우 경고 메시지 출력 후 건너뛰기
                    if "UNIQUE constraint failed: products_images.image_url" in str(e):
                        # URL의 일부만 출력하여 로그 길이를 줄임
                        print(f"  [DB 경고] 동일한 이미지 URL이 이미 존재하여 건너뜁니다. (URL: {url[:50]}...)") 
                    else:
                        print(f"  [DB 경고] 예상치 못한 중복 발생: {final_filename}. 건너뜁니다.")

                except sqlite3.Error as db_e:
                    print(f"  [DB 오류] 데이터 저장 실패 (카테고리: {category_name_kor}, 파일: {final_filename}): {db_e}")

    except (KeyError, TypeError, IndexError) as e:
        print(f"[오류] 데이터 구조를 파싱하는 중 오류 발생 (카테고리: {category_name_kor}): {e}")
        return 0 

    return items_processed_success

# 메인 크롤링 함수
def main_crawl():

    # SQLite 데이터베이스 연결 및 설정
    conn, cursor = setup_database(DATABASE_NAME)
    if not conn or not cursor:
        return # DB 연결 실패 시 종료

    # 크롤링할 모든 카테고리를 리스트로 통합
    categories_to_crawl = []
    # (카테고리 ID, 카테고리 이름, columns, max_filters) 튜플로 저장
    for code, name in LARGE_IMAGE_CATEGORIES.items():
        categories_to_crawl.append((code, name, 2, 5))
        
    for code, name in COMPACT_PRODUCT_CATEGORIES.items():
        categories_to_crawl.append((code, name, 4, 3))
        
    print(f"\n총 {len(categories_to_crawl)}개의 카테고리 크롤링을 시작합니다.")
    
    total_crawled_count = 0
    
    # 1. 카테고리별 루프
    for category_id, category_name_kor, columns, max_filters in categories_to_crawl:
        
        print(f"\n\n==================== 📦 카테고리 시작: {category_name_kor} ({category_id}) | Columns: {columns}, Filters: {max_filters} ====================")

        # 카테고리별 초기화
        crawled_count_in_category = 0 
        total_products = float('inf') 
        current_page = 1
        
        # 페이로드 템플릿 복사 및 카테고리 설정 업데이트
        payload = BASE_PAYLOAD_TEMPLATE.copy()
        payload['searchParameters']['input'] = category_id
        payload['components'][0]['columns'] = columns
        payload['components'][0]['filterConfig']['max-num-filters'] = max_filters
        
        # 2. 페이지네이션 반복 (Inner Loop)
        while crawled_count_in_category < total_products:
            offset = (current_page - 1) * PRODUCT_SIZE
            # print(f"\n--- 📄 {category_name_kor} - 페이지 {current_page} (Offset: {offset}) 크롤링 시작 ---") # 너무 자주 출력되어 주석 처리

            # Offset 업데이트
            payload['components'][0]['window']['offset'] = offset

            try:
                # API 요청
                api_resp = requests.post(API_URL, json=payload)
                api_resp.raise_for_status()
                product_data = api_resp.json()

                # 첫 요청 시 총 상품 수 업데이트
                if current_page == 1:
                    first_result = product_data.get('results', [{}])[0]
                    total_products = first_result.get('metadata', {}).get('max', 0)
                    
                    max_pages = (total_products + PRODUCT_SIZE - 1) // PRODUCT_SIZE if total_products > 0 else 1
                    print(f"*** {category_name_kor} | 전체 상품 개수: {total_products}개 (예상 최대 페이지 수: {max_pages}) ***")
                    if total_products == 0:
                        print(f"  [알림] 상품이 존재하지 않아 다음 카테고리로 이동합니다.")
                        break 

                # 상품 목록 처리 및 데이터베이스 저장
                items_on_page = process_product_data(product_data, cursor, category_id, category_name_kor)

                # 페이지별로 DB에 변경사항 커밋
                conn.commit()

                # API가 빈 목록을 반환하는 경우 무한 루프에 빠지지 않도록 처리
                if items_on_page == 0 and offset > 0 and crawled_count_in_category == total_products:
                    # 이 페이지에서 0개를 처리했지만, 이미 total_products만큼 처리했다면 종료
                    break
                elif items_on_page == 0 and offset > 0:
                     # 이 페이지에서 0개를 처리했지만, 아직 total_products를 다 처리하지 못했다면
                     # 데이터 끝에 도달했을 가능성이 높으므로 종료
                    print(f"  [알림] 더 이상 반환되는 상품이 없어 {category_name_kor} 크롤링을 종료합니다.")
                    break


                # 크롤링된 상품 수 업데이트
                crawled_count_in_category += items_on_page
                total_crawled_count += items_on_page

                # 다음 페이지로 이동
                current_page += 1
                
                # API 서버 부하 방지를 위해 잠시 대기
                time.sleep(1) 

            except requests.exceptions.RequestException as e:
                print(f"\n[치명적 오류] API 요청 실패 (카테고리: {category_name_kor}, 페이지 {current_page}): {e}")
                break
            except Exception as e:
                print(f"\n[치명적 오류] 알 수 없는 오류 발생 (카테고리: {category_name_kor}, 페이지 {current_page}): {e}")
                break
        
        print(f"==================== ✅ 카테고리 종료: {category_name_kor} | 크롤링된 상품 수: {crawled_count_in_category} ====================")


    # 크롤링 완료 후 DB 연결 종료
    print(f"\n\n==================== ✨ 전체 크롤링 완료. 총 {total_crawled_count}개 상품 데이터 저장됨. ====================")
    conn.close()

if __name__ == "__main__":
    main_crawl()